# Lab 11: Integer Linear programming and Dinamic Programming 
In this lab, we will see Integer Linear programming and Dynamic programming (DP). 

For the Integer Linear Programming, you will proceed as in the previous lab, writing some models and solving the problem. For Dynamic Programming, we will ask you to solve the knapsack problem with DP. 

In [ ]:
!pip install -q condacolab
import condacolab

condacolab.install()
!conda install pyscipopt
!pip install scikit-optimize
!pip install treed
!pip install PrettyPrintTree

In [ ]:
import itertools as it
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pyscipopt
import seaborn as sns

from pyscipopt import Model, Eventhdlr, SCIP_EVENTTYPE
from pyscipopt import Model, quicksum
from PrettyPrint import PrettyPrintTree

In [ ]:
#@title ILP implementation

class LPstatEventhdlr(Eventhdlr):
    """PySCIPOpt Event handler to collect data on LP events."""

    transvars = {}

    def collectNodeInfo(self, firstlp=True):
        objval = self.model.getSolObjVal(None)

        LPsol = {}
        if self.transvars == {}:
            self.transvars = self.model.getVars(transformed=True)
        for var in self.transvars:
            solval = self.model.getSolVal(None, var)
            LPsol[var.name] = self.model.getSolVal(None, var)

        # skip duplicate nodes
        # if self.nodelist and LPsol == self.nodelist[-1].get("LPsol"):
        #     return
        node = self.model.getCurrentNode()
        if node.getNumber() != 1:
            parentnode = node.getParent()
            parent = parentnode.getNumber()
        else:
            parent = 1
        depth = node.getDepth()
        age = self.model.getNNodes()
        condition = math.log10(self.model.getCondition())
        iters = self.model.lpiGetIterations()
        pb = self.model.getPrimalbound()
        if pb >= self.model.infinity():
            pb = None

        nodedict = {
            "number": node.getNumber(),
            "LPsol": LPsol,
            "objval": objval,
            "parent": parent,
            "age": age,
            "depth": depth,
            "first": firstlp,
            "condition": condition,
            "iterations": iters,
            # "variables": self.model.getNVars(),
            # "constraints": self.model.getNConss(),
            "rows": self.model.getNLPRows(),
            "primalbound": pb,
            "dualbound": self.model.getDualbound(),
            "time": self.model.getSolvingTime()
        }

        self.nodelist.append(nodedict)

    def eventexec(self, event):

        if event.getType() == SCIP_EVENTTYPE.FIRSTLPSOLVED:
            self.collectNodeInfo(firstlp=True)
        elif event.getType() == SCIP_EVENTTYPE.LPSOLVED:
            self.collectNodeInfo(firstlp=False)
        else:
            print("unexpected event:" + str(event))
        return {}

    def eventinit(self):
        self.model.catchEvent(SCIP_EVENTTYPE.LPEVENT, self)


def convertSolToDict(sol):
    ssol = str(sol)
    items = ssol[1:-1].split(', ')
    res = {}
    for item in items:
        [k, v] = item.split(':')
        res[k.lstrip('\'t_').rstrip('\'')] = int(float(v))

    return res


def solve_model(model):
    model.setPresolve(pyscipopt.SCIP_PARAMSETTING.OFF)
    model.setHeuristics(pyscipopt.SCIP_PARAMSETTING.OFF)
    model.disablePropagation()
    #model.redirectOutput()
    nodelist = []
    eventhdlr = LPstatEventhdlr()
    eventhdlr.nodelist = nodelist
    model.includeEventhdlr(
        eventhdlr, "LPstat", "generate LP statistics after every LP event"
    )
    model.optimize()
    frontier_history = []
    for nd in nodelist:
        frontier_history.append((convertSolToDict(nd['LPsol']), nd['objval']))
    best_sol = model.getBestSol()
    best_val = model.getObjVal()
    return convertSolToDict(best_sol), best_val, frontier_history

# Ex. 0

This is an example to show how the Library works

---

Slack form:

minimize  $2x_1 + x_2 − 2x_3$

subject to 

- $0.7x_1 + 0.5x_2 +x_3 \geq 1.8$
- $x_i \in [0,1]\ \forall i$

In [ ]:
model = Model("")
v = [model.addVar(vtype='i', name=str(i), lb=0, ub=1) for i in range(3)]
c = [0.7, 0.5, 1]
model.addCons(quicksum(c[i] * v[i] for i in range(3)) >= 1.8)
sum([i for i in range(3)])
model.setObjective(2 * v[0] + v[1] - 2 * v[2], "minimize")
best_sol, best_val, partial_frontier_history = solve_model(model)

print(best_sol)
print(best_val)
print(partial_frontier_history)

# Ex. The N-queens Problem | ILP

The N queens puzzle is the problem of placing $N$ chess queens on an $N \times N (N \geq 4)$ chessboard so that no two queens threaten each other; thus, a solution requires that no two queens share the same row, column, or diagonal. Try different values of $N$ and shows how the problem complexity increases.

---

Foreach cell in the chessboard a variable

Each cell can be either 0 or 1, where
- 0 means the cell is empty
- 1 means the cell contains a queen

We can encode the constrains on rows, columns and diagonals as a disequation, where the sum of the corresponding cells must be less or equal to 1

less or equal because if it is true that there must be a queen foreach row and column, the same does not apply for the diagonals

The problem becomes a maximization problem on the sum of every cells


In [ ]:
n = 10

chessboard = np.asarray([
    np.asarray([
        f'({r} {c})'
        for c
        in range(n)
    ])
    for r
    in range(n)
])

print(chessboard)

print('-' * 90)

for r in range(0, n):
    print(chessboard[r])

print('-' * 90)

for c in range(0, n):
    print(chessboard[:, c])

print('-' * 90)

for d in range(-n + 1, n):
    print(chessboard.diagonal(d))

print('-' * 90)

for d in range(-n + 1, n):
    print(np.fliplr(chessboard).diagonal(d))

print('-' * 90)

print(np.reshape(chessboard, -1))

In [ ]:
n = 10

model = Model('')
chessboard = np.asarray([
    np.asarray([
        model.addVar(vtype='i', name=f'({r} {c})', lb=0, ub=1)
        for c
        in range(n)
    ])
    for r
    in range(n)
])

# rows
for r in range(n):
    model.addCons(quicksum(chessboard[r]) <= 1)

# columns
for c in range(n):
    model.addCons(quicksum(chessboard[:, c]) <= 1)

# diagonals top left - bottom right
for d in range(-n + 1, n):
    model.addCons(quicksum(chessboard.diagonal(d)) <= 1)

# diagonals top right - bottom left
for d in range(-n + 1, n):
    model.addCons(quicksum(np.fliplr(chessboard).diagonal(d)) <= 1)

model.setObjective(quicksum(np.reshape(chessboard, -1)), "maximize")
best_sol, best_val, partial_frontier_history = solve_model(model)

_, ax = plt.subplots(1, figsize=(10, 10))

sns.heatmap(
    np.asarray([
        np.asarray([
            1 - (r + c) % 2
            for c
            in range(n)
        ])
        for r
        in range(n)
    ]),
    linewidth=0.5,
    cbar=False,
    xticklabels=[],
    yticklabels=[],
    ax=ax,
)

queens = np.asarray([(r, c) for (r, c) in it.product(range(n), range(n)) if best_sol.get(f'({r} {c})') == 1])
ax.scatter(queens[:, 0] + 0.5, queens[:, 1] + 0.5, c='C3', marker='h', s=1000)
plt.show()

print(queens)
print(best_val)
# print(partial_frontier_history)

# Traveling salesman problem | ILP

The goal of the TSP is to find the shortest Hamiltonian cycle (a cycle that visits each node only once) on a graph of N nodes. Solve the ILP problem and visualize a solution.

In [ ]:
# graph is a matrix
# minimize the sum of used edges cost
# visit each node at most once ~ find the cycle
#   -> foreach node u, the sum of the edges (u, _) must be equal to 1
#   -> foreach node v, the sum of the edges (_, v) must be equal to 1
# disallow sub-cycles
#   -> u_j - u_i + (n + 1) * x_ij <= n

n = 15

rnd = np.random.default_rng(seed=0)
costs = rnd.integers(low=-370, high=100, size=(n, n))
np.fill_diagonal(costs, 0)

DG = nx.DiGraph()
DG.add_nodes_from(range(n))
DG.add_weighted_edges_from([
    (a, b, costs[a][b])
    for (a, b)
    in it.product(range(n), range(n))
    if costs[a][b] > 0
])

_, ax = plt.subplots(1, figsize=(12, 12))
pos = nx.circular_layout(DG)
nx.draw_networkx(
    G=DG,
    pos=pos,
    ax=ax,
    cmap=plt.cm.magma,
    node_size=2e3,
    alpha=0.5,
    node_color='C3',
    with_labels=True,
)
plt.show()

In [ ]:
model = Model('')
edges = np.asarray([
    np.asarray([
        model.addVar(vtype='i', name=f'{a} -> {b}', lb=0, ub=1) if costs[a][b] > 0 else None
        for b
        in range(n)
    ])
    for a
    in range(n)
])

# foreach node u, the sum of the edges (u, _) must be equal to 1
for u in range(n):
    model.addCons(quicksum(e for e in edges[u] if e is not None) == 1)

# foreach node v, the sum of the edges (_, v) must be equal to 1
for v in range(n):
    model.addCons(quicksum(e for e in edges[:, v] if e is not None) == 1)

# u_j - u_i + (n - 1) * x_ij <= (n - 2)
# us = np.asarray([
#     model.addVar(vtype='i', name=f'u_{i}', lb=0)
#     for i 
#     in range(n)
#     ])
# for i in range(1, n):
#   for j in range(1, n):
#     if (edges[i][j] is not None):
#       model.addCons(us[j] - us[i] + (n - 1) * edges[i][j] <= (n - 2))

model.setObjective(quicksum(
    c * e
    for (e, c)
    in zip(np.reshape(edges, -1), np.reshape(costs, -1))
    if e is not None
), "minimize")
best_sol, best_val, partial_frontier_history = solve_model(model)

In [ ]:
def show(solution: dict):
    def planar(ax):
        DG = nx.DiGraph()
        DG.add_nodes_from(range(n))
        DG.add_weighted_edges_from([
            (a, b, costs[a][b])
            for (a, b)
            in it.product(range(n), range(n))
            if costs[a][b] > 0
            if solution.get(f'{a} -> {b}') == 1
        ])

        pos = nx.planar_layout(DG)
        nx.draw_networkx(
            G=DG,
            pos=pos,
            ax=ax,
            cmap=plt.cm.magma,
            node_size=2e3,
            alpha=0.5,
            node_color='C3',
            with_labels=True,
            edgelist=[],
        )
        nx.draw_networkx_edges(
            G=DG,
            pos=pos,
            ax=ax,
            width=3,
            alpha=1,
            arrowstyle="->",
            edge_color=['C8'] + list(it.repeat('C3', n - 1)),
            edge_cmap=plt.cm.magma,
        )
        nx.draw_networkx_edge_labels(
            G=DG,
            pos=pos,
            ax=ax,
            label_pos=0.7,
            edge_labels=nx.get_edge_attributes(DG, 'weight'),
        )

    def circular(ax):
        DG = nx.DiGraph()
        DG.add_nodes_from(range(n))
        es = [
            (a, b, costs[a][b], solution.get(f'{a} -> {b}') == 1)
            for (a, b)
            in it.product(range(n), range(n))
            if costs[a][b] > 0
        ]
        for (a, b, w, c) in es:
            if c:
                DG.add_edge(a, b, weight=costs[a][b])
            else:
                DG.add_edge(a, b)

        pos = nx.circular_layout(DG)
        nx.draw_networkx(
            G=DG,
            pos=pos,
            ax=ax,
            cmap=plt.cm.magma,
            node_size=2e3,
            alpha=0.5,
            node_color='C3',
            with_labels=True,
            edgelist=[(a, b) for (a, b, _, c) in es if not c],
        )
        nx.draw_networkx_edges(
            G=DG,
            pos=pos,
            ax=ax,
            width=3,
            alpha=1,
            edgelist=[(a, b) for (a, b, _, c) in es if c],
            arrowstyle="->",
            edge_color=['C8'] + list(it.repeat('C3', n - 1)),
            edge_cmap=plt.cm.magma,
        )
        nx.draw_networkx_edge_labels(
            G=DG,
            pos=pos,
            ax=ax,
            label_pos=0.7,
            edge_labels=nx.get_edge_attributes(DG, 'weight'),
        )

    fig, (ax_circular, ax_planar) = plt.subplots(1, 2, figsize=(24, 12))
    ax_circular.set_axis_off()
    ax_planar.set_axis_off()
    circular(ax_circular)
    planar(ax_planar)
    fig.subplots_adjust(wspace=0, hspace=0)
    plt.show()


show(best_sol)

# Ex. KNAPSACK PROBLEM | DP

Solve the Knapsack problem using DP.
You can use the Class from lab 5.

Run the solution multiple time, and change the total capacity and the number of objects and shows how the number of sub-problems changes.

In [ ]:
class Knapsack_0_1:

    def __init__(self):
        self._items = [
            []
        ]
        self._BAG_CAPACITY = 10
        self.history = []
        self.values = []

    def _get_value(self, solution):
        cur_cap = self._BAG_CAPACITY
        cur_val = 0
        for i, v in enumerate(solution):
            if v == 1:
                cur_val += self._items[i]['value']
                cur_cap -= self._items[i]['volume']
            if cur_cap < 0:
                return 0
        return -cur_val

    def __call__(self, solution):
        value = self._get_value(solution)
        self.history.append(solution)
        self.values.append(value)
        return value

    def trend(self):
        plt.figure()
        plt.plot(self.values)
        plt.show()


In [ ]:
class Tree:
    def __init__(self, val: str, children: list):
        self.val = val 
        self.children = children

def tree2html(tree: Tree) -> str:
    return f'<li>{tree.val}</li>' if not tree.children else f'<li><details open><summary>{tree.val}</summary><ul>\n{"".join([ tree2html(c) for c in tree.children ])}\n</ul></details></li>'

prefix = '''
<!DOCTYPE html>
<html>
<head>
<style>
html {
  font-family: monospace;
  font-size: 1.5em;
  line-height: 1.5;
}

.tree{
  --spacing : 1.5rem;
  --radius  : 10px;
}

.tree li{
  display      : block;
  position     : relative;
  padding-left : calc(2 * var(--spacing) - var(--radius) - 2px);
}

.tree ul{
  margin-left  : calc(var(--radius) - var(--spacing));
  padding-left : 0;
}

.tree ul li{
  border-left : 2px solid #ddd;
}

.tree ul li:last-child{
  border-color : transparent;
}

.tree ul li::before{
  content      : '';
  display      : block;
  position     : absolute;
  top          : calc(var(--spacing) / -2);
  left         : -2px;
  width        : calc(var(--spacing) + 2px);
  height       : calc(var(--spacing) + 1px);
  border       : solid #ddd;
  border-width : 0 0 2px 2px;
}

.tree summary{
  display : block;
  cursor  : pointer;
}

.tree summary::marker,
.tree summary::-webkit-details-marker{
  display : none;
}

.tree summary:focus{
  outline : none;
}

.tree summary:focus-visible{
  outline : 1px dotted #000;
}

.tree li::after,
.tree summary::before{
  content       : '';
  display       : block;
  position      : absolute;
  top           : calc(var(--spacing) / 2 - var(--radius));
  left          : calc(var(--spacing) - var(--radius) - 1px);
  width         : calc(2 * var(--radius));
  height        : calc(2 * var(--radius));
  border-radius : 50%;
  background    : #ddd;
}

.tree summary::before{
  content     : '+';
  z-index     : 1;
  background  : #696;
  color       : #fff;
  line-height : calc(2 * var(--radius) - 2px);
  text-align  : center;
}

.tree details[open] > summary::before{
  content    : '-';
  background : chocolate;
}
</style>
</head>
<body>
<ul class="tree">

'''

suffix = '''

</ul>
</body>
</html>
'''

for n in [15]:
    for c in range(10, 21, 5):

        rnd = np.random.default_rng(seed=n)
        knapsack = rnd.integers(low=1, high=20, size=(n, 2))


        def value(i: int) -> int:
            return knapsack[i][0]


        def weight(i: int) -> int:
            return knapsack[i][1]


        def solveREC() -> tuple:

            mem = np.full((n, c + 1), np.nan)

            def solve(i: int, c: int) -> tuple:

                current = f'{c} / [A..{chr(ord("A") + i)}]' if i > 1 else f'{c} / [A, B]' if i == 1 else f'{c} / [A]' if i == 0 else f'{c} / []'

                if c < 0:
                    return (float('-inf'), Tree(current + f'\n{float("-inf")}', []))

                if c == 0 or i < 0:
                    return (0, Tree(current + f'\n{0}', []))

                if np.isnan(mem[i, c]):
                    (tv, tn) = solve(i - 1, c - weight(i))
                    tv += value(i)
                    (lv, ln) = solve(i - 1, c)

                    if tv >= lv:
                      mem[i, c] = tv
                      return (mem[i, c], Tree(current + f' : max ( {tv - value(i)} + {value(i)} , {lv} )', [tn, ln]))
                    else:
                      mem[i, c] = lv
                      return (mem[i, c], Tree(current + f' : max ( {tv - value(i)} + {value(i)} , {lv} )', [tn, ln]))

                return (mem[i, c], Tree(current + f' : {mem[i, c]}', [Tree('*', [])]))

            return solve(n - 1, c)


        def solveIT():

            mem = np.zeros((n, c + 1))
            chosen = np.full((n, c + 1), False)

            for i in range(n):
                for k in range(c + 1):
                    if weight(i) > k:
                        mem[i, k] = mem[i - 1, k]
                    else:
                        if value(i) + mem[i - 1, k - weight(i)] > mem[i - 1, k]:
                            mem[i, k] = value(i) + mem[i - 1, k - weight(i)]
                            chosen[i, k] = True
                        else:
                            mem[i, k] = mem[i - 1, k]

            k = c
            xs = []
            for i in reversed(range(n)):
                if chosen[i, k]:
                    xs.append(i)
                    k -= weight(i)

            return (mem[n - 1, c], xs)


        print(knapsack)
        print(solveIT())
        (v, r) = solveREC()
        # np.savetxt(f'out {n} {c}', [PrettyPrintTree(lambda x: x.children, lambda x: x.val, default_orientation=PrettyPrintTree.HORIZONTAL, return_instead_of_print=True, color=None)(r)], fmt='%s')
        np.savetxt(f'out_{n}_{c}.html', [prefix, tree2html(r), suffix], fmt='%s')